In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import numpy as np
import tensorflow as tf
if __name__ == "__main__":
    sys.path.append("../../")
from make_anchors_orig import make_anchors
from notebooks_src.configs import configs 
from notebooks_src.box_processing.tf_box_util import convert_to_xyminmax, make_box_coords_relative,\
encode_to_scaled_offsets, convert_to_yxhw,make_actual_gt_box_mask, zero_out_negatives, preprocess_box_coordinates
import numpy as np
from make_anchors_orig import make_anchors_dict


importing Jupyter notebook from make_anchors_orig.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/box_encode_decode_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/tensorboard_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/fit_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/labels_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/load_data_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/losses_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/metrics_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/models_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/configs/optimizers_configs.ipynb
importing Jupyter notebook from ../../notebooks_src/box_processing/tf_box_util.ipynb


In [2]:
def encode(bboxes,labels):    
    anchors_dict = make_anchors_dict()
    encoded_boxes_labels_dict = {fmap_shape: {"boxes": _encode_boxes(bboxes, anchors),
                                              "labels":_encode_labels(labels, anchors)} 
                                 for fmap_shape, anchors in anchors_dict.iteritems()}

    return encoded_boxes_labels_dict  

In [4]:
def _encode_labels(labels, anchors):
    ay, ax, ah, aw = anchors
    encoded_labels = labels * tf.cast(tf.expand_dims(tf.expand_dims(tf.ones_like(ay*ah),axis=-1),axis=-1), dtype=tf.int32)
    encoded_labels = tf.transpose(encoded_labels,perm=[3,0,1,2,4])
    # eoncoded_labels is (batch_size, y, x, num_anchors, max_boxes(15))
    return encoded_labels

In [5]:
def expand_anchors(anchors):
    ay, ax, ah, aw = anchors
    ay, ax = [np.expand_dims(tens, axis=-1) for tens in [ay,ax]]
    return ay,ax,ah,aw
    

In [6]:
def _encode_boxes(bboxes, anchors):
    ymin,xmin, ymax, xmax = preprocess_box_coordinates(bboxes)
    cy,cx,h,w = convert_to_yxhw(ymin, ymax,xmin, xmax)
    ay, ax, ah, aw = expand_anchors(anchors)
    enc_y, enc_x, enc_h, enc_w = encode_to_scaled_offsets(cy,cx,h,w, ay, ax, ah, aw)
    
    encoded_boxes = tf.stack([enc_y, enc_x, enc_h, enc_w],axis=-1)
    encoded_boxes = tf.transpose(encoded_boxes,perm=(2,0,1,4,5,3))
    # encoded_boxes is (batch_size, y, x, num_anchors, num_coordinates(4), max_boxes(15))
    return encoded_boxes
    
    

In [6]:
if __name__ == "__main__":
    import h5py
    bboxes = h5py.File(configs["tr_data_file"])["boxes"][7:12]
    bbox_coords = bboxes[:,:,:4]
    labels = bboxes[:,:,4]
    ymin = encode(bbox_coords, labels)
    
    with tf.Session() as sess:
        print sess.run(ymin)
        assert False
        loc_tens = sess.run(btens)
        lab_tens = sess.run(ltens)
        for k, v in loc_tens.iteritems():
            print k
            print v

({(6, 9): array([[[[[[ 0.83512932, -0.21551725, -0.21551725, ..., -0.21551725,
            -0.21551725, -0.21551725],
           [ 1.03380919, -0.21551724, -0.21551724, ..., -0.21551724,
            -0.21551724, -0.21551724],
           [-2.22786093,        -inf,        -inf, ...,        -inf,
                   -inf,        -inf],
           [-2.22786117,        -inf,        -inf, ...,        -inf,
                   -inf,        -inf]],

          [[ 0.739353  , -0.19080079, -0.19080079, ..., -0.19080079,
            -0.19080079, -0.19080079],
           [ 0.9152475 , -0.19080079, -0.19080079, ..., -0.19080079,
            -0.19080079, -0.19080079],
           [-2.34967208,        -inf,        -inf, ...,        -inf,
                   -inf,        -inf],
           [-2.34967208,        -inf,        -inf, ...,        -inf,
                   -inf,        -inf]],

          [[ 1.18105125, -0.30478743, -0.30478743, ..., -0.30478743,
            -0.30478743, -0.30478743],
           [ 0

AssertionError: 